In [ ]:
!pip install sentence-transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
languages = {
    'arq':'Algerian Arabic',
    'ary': 'Moroccan Arabic',
    'eng':'English',
    'kin':'Kinyarwanda',
    'hau':'Hausa',
    'mar':'Marathi',
    'tel': 'Telugu'
}
model_name='sentence-transformers/LaBSE'

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util, models
from torch.utils.data import DataLoader
import torch
import numpy as np
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
import zipfile
import os

In [ ]:
train_df=pd.read_csv(f'https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/tel/tel_train.csv')
dev_df=pd.read_csv(f'https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/tel/tel_dev_with_labels.csv')
test_df=pd.read_csv(f'https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/tel/tel_test.csv')


In [ ]:
train_df[['text1', 'text2']] = train_df['Text'].str.split('\n', n=1, expand=True) # Splitting into text1 and text2
train_df.drop('Text', axis=1, inplace=True) #Removing Text column, as it is redundant

In [ ]:
dev_df[['text1', 'text2']] = dev_df['Text'].str.split('\n',n=1, expand=True) # Splitting into text1 and text2
dev_df.drop('Text', axis=1, inplace=True) #Removing Text column, as it is redundant

In [ ]:
train_df.head()

,PairID,Score,text1,text2
0,TEL-train-00001,0.88,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...","""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడిన ఘటన..."
1,TEL-train-00002,0.87,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...","""కన్నతండ్రి కూతురుపై అత్యాచారం చేసిన దారుణ ఘటన..."
2,TEL-train-00003,0.87,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...","""గంగానదిని రక్షించేందుకు ప్రత్యేక చట్టం తేవాలన..."
3,TEL-train-00004,0.84,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...","""అమెరికా ప్రభుత్వానికి కరోనా వైరస్పై జరుగుతున్..."
4,TEL-train-00005,0.84,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...","""కరోనావైరస్ (కోవిడ్-19) మానవులకు మరియు జంతువుల..."


In [ ]:
train_df.shape

(1170, 4)

In [ ]:
dev_df.head()

,PairID,Score,text1,text2
0,TEL-dev-00001,0.65,"""బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండోరో...","""బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా గారు ఈ రోజు ..."
1,TEL-dev-00002,0.60,"""ఈ ఏడాది జనవరి 1వ తేదీ నాటికి ఢిల్లీలో 10,76,4...","""ఈ ఏడాది చివరి నాటికల్లా కంపెనీ ఈ అప్పుల్లో కొ..."
2,TEL-dev-00003,0.62,"""ప్రతిపక్షాలు అనవసర రాద్ధాంతం చేస్తున్నాయి : హ...","""హైదరాబాద్ : తెలంగాణ అభివృద్ధిని అడ్డుకునేందుక..."
3,TEL-dev-00004,0.57,"""హైదరాబాద్ : కాంగ్రెస్ పార్టీ అధికారంలోకి రాగా...","""ఇందిరమ్మ ఇళ్లలో అక్రమాలు జరిగాయని టిఆర్ఎస్ నా..."
4,TEL-dev-00005,0.53,"""ప్రజారాజధానిగా రూపుదిద్దేందుకు అవసరమైన పెట్టు...","""ఈ ఒప్పం దం తక్షణమే అమల్లోకి వస్తుందని సంబంధిత..."


In [ ]:
dev_df.shape

(130, 4)

In [ ]:
train_df = pd.concat([train_df, dev_df], ignore_index=True)

In [ ]:
# train_df, val_df = train_df,dev_df

In [ ]:
train_df.shape

(1300, 4)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:

# word_embedding_model = models.Transformer(model_name)


# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
#                                pooling_mode_mean_tokens=True,
#                                pooling_mode_cls_token=False,
#                                pooling_mode_max_tokens=False)

# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [ ]:
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
train_examples = [
    InputExample(texts=[row["text1"], row["text2"]], label=row["Score"])
    for i, row in train_df.iterrows()
]

In [ ]:
train_dataloader = DataLoader(train_examples, batch_size=8, shuffle=True)  # Adjust batch_size as needed

In [ ]:
train_loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
def compute_spearman_correlation(predictions, targets):
    return spearmanr(predictions, targets).correlation

def on_epoch_end(epoch):
    val_scores = []  # Empty list to store all predicted scores

    for i, row in val_df.iterrows():
        text1 = row["text1"]
        text2 = row["text2"]

        # Generate individual embeddings
        embedding1 = model.encode([text1])
        embedding2 = model.encode([text2])

        # Calculate and append score
        score = util.cos_sim(embedding1, embedding2)[0][0]  # Access specific element
        val_scores.append(np.round(float(score.item()), 2))

    print(f"Spearman Coefficient on {epoch}th epoch: {spearmanr(val_scores, val_df['Score']).correlation}")

In [ ]:
num_epochs = 20
on_epoch_end(0)
for epoch in range(1,num_epochs+1):
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=100,
    )
    on_epoch_end(epoch)

Spearman Coefficient on 0th epoch: 0.7715653485702082


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/130 [00:00<?, ?it/s]

Spearman Coefficient on 1th epoch: 0.8001713804577834


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/130 [00:00<?, ?it/s]

Spearman Coefficient on 19th epoch: 0.7642987145519842


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/130 [00:00<?, ?it/s]

Spearman Coefficient on 20th epoch: 0.765323900536878


In [ ]:
test_df.shape

(297, 2)

In [ ]:

test_df[['text1', 'text2']] = test_df['Text'].str.split('\n', n=1, expand=True) # Splitting into text1 and text2
test_df.drop('Text', axis=1, inplace=True) #Removing Text column, as it is redundant

In [ ]:
all_scores = []  # Empty list to store all predicted scores

for i, row in test_df.iterrows():
    text1 = row["text1"]
    text2 = row["text2"]

    # Generate individual embeddings
    embedding1 = model.encode([text1])
    embedding2 = model.encode([text2])

    # Calculate and append score
    score = util.cos_sim(embedding1, embedding2)[0][0]  # Access specific element
    all_scores.append(np.round(float(score.item()), 2))

In [ ]:
test_df["Pred_Score"] = np.round(all_scores, 2)

In [ ]:
test_df = test_df[['PairID','Pred_Score']]

In [ ]:
test_df

,PairID,Pred_Score
0,TEL-test-00001,0.54
1,TEL-test-00002,0.24
2,TEL-test-00003,0.47
3,TEL-test-00004,0.48
4,TEL-test-00005,0.36
...,...,...
292,TEL-test-00293,0.42
293,TEL-test-00294,0.38
294,TEL-test-00295,0.49
295,TEL-test-00296,0.66


In [ ]:
test_df.to_csv(f'pred_tel_a.csv', index=False)

In [ ]:
csv_file_name = f'pred_tel_a.csv'
zip_file_name = f'pred_tel_a.zip'

with zipfile.ZipFile(zip_file_name, 'w') as zip_file:
    zip_file.write(csv_file_name, os.path.basename(csv_file_name))

In [ ]:
model.save(f"models/finetuned_{name}_model")